# Check results and make plots on individual cells
- Find cell index based on terrain plots in a map
- Find all neighborhood cells based on an input lat/lon
- etc

## get the `pyDAmonitor_ROOT` env variable
This step is highly recommended. It is required if one want to use the DAmonitor Python package or use the MPAS/FV3 sample data or local cartopy nature_earth_data.

In [ ]:
%%time
# autoload external python modules if they changed
%load_ext autoreload
%autoreload 2
    
import sys, os
pyDAmonitor_ROOT=os.getenv("pyDAmonitor_ROOT")
if pyDAmonitor_ROOT is None:
    print("!!! pyDAmonitor_ROOT is NOT set. Run `source ush/load_pyDAmonitor.sh`")
else:
    print(f"pyDAmonitor_ROOT={pyDAmonitor_ROOT}\n")
sys.path.insert(0, pyDAmonitor_ROOT)

## import modules

In [ ]:
%%time
import numpy as np
from netCDF4 import Dataset

import matplotlib.pyplot as plt
import matplotlib as mpl
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from DAmonitor.base import query_dataset, query_obj
cartopy.config['data_dir'] = f"{pyDAmonitor_ROOT}/data/natural_earth_data"

## read in `invariant.nc`

In [ ]:
ds0 = Dataset(os.path.join(pyDAmonitor_ROOT,'data/mpasjedi/invariant.nc'), 'r')
query_dataset(ds0)

## check `indexToCellID`, attributes, dimensions, etc

In [ ]:
print(ds0["indexToCellID"][:])
# print(ds0.ncattrs())
print(ds0.dimensions.keys())
print(ds0.dimensions["nVertLevels"].size)
print(ds0.dimensions["nCells"].size)
ds0.dimensions
# ds0

## check `zgrid`, `ter`

In [ ]:
print(ds0["zgrid"].shape)
print(ds0["zgrid"][2341817,:])
print(ds0["zgrid"][50,:])
print(ds0["ter"][2341817])
print(ds0["ter"][50])
ds0["zgrid"]

## compute a new zgrid by removing the terrain height

In [ ]:
%%time
agl=np.empty_like(ds0["zgrid"][:])
for i in range(60):
    agl[:,i]=ds0["zgrid"][:,i]-ds0["ter"][:]

In [ ]:
np.set_printoptions(suppress=True)   # disables scientific notation
np.set_printoptions(precision=3)     # optional: 2 decimal places
print(agl[2341817,:])
print(agl[50,:])    # the AGL zgrid values are different at different locations

## scatter all cell terrain values and when a mouse hovers on a cell, show its `lat`,`lon`,`ter` and `iCell` (cell index)
Write down iCell and then we can print a field profile for that cell

In [ ]:
import plotly.express as px
import xarray as xr

# plot the whole map
factor=50  # plotting all cells take too many memories, so plot every factor cells
lon = np.degrees(ds0.variables['lonCell'][::factor])
lat = np.degrees(ds0.variables['latCell'][::factor])
iCell = ds0.variables['indexToCellID'][::factor]
ter = ds0["ter"][::factor]

# plot a subdomain defined by latmin, latmax, lonmin, lonmax
# latmin, latmax = 37, 41
# lonmin0, lonmax0 = -109, -102
# #
# lon0 = np.degrees(ds0.variables['lonCell'][:])
# lat0 = np.degrees(ds0.variables['latCell'][:])
# lonmin = lonmin0 + 360
# lonmax = lonmax0 + 360
# mask = (lat0 >= latmin) & (lat0 <= latmax) & (lon0 >= lonmin) & (lon0 <= lonmax)
# lat = lat0[mask]
# lon = lon0[mask]
# iCell = ds0.variables['indexToCellID'][:][mask]
# ter = ds0.variables["ter"][:][mask]

# use projections, but LLC is not natively supported in plotly
# fig = px.scatter_geo(
#     lat=lat,
#     lon=lon,
#     color=ter,
#     color_continuous_scale="Viridis",  # choose any supported colorscale
#     projection="mercator",
#     hover_name=None,                   # optional: show info on hover
#     hover_data={"lat": lat, "lon": lon, "ter": ter},
#     size_max=10,
# )

# use maps
fig = px.scatter_map(
    lat=lat,
    lon=lon,
    color=ter,
    color_continuous_scale="Viridis",  # choose any supported colorscale
    hover_name=None,                   # optional: show info on hover
    hover_data={"lat": lat, "lon": lon, "ter": ter, "iCell": iCell},
    size_max=1,
)

# set width and height
fig.update_layout(
    width=1000,   # pixels
    height=800,   # pixels
    title=""
)

# Show interactive map
fig.show()

## Print pressure profile in the Atlantic at iCell=1702401

In [ ]:
iCell = 1702401

ds1 = Dataset(os.path.join(pyDAmonitor_ROOT,'data/mpasjedi/bkg.nc'), 'r')
query_dataset(ds1)
pfull = ds1['pressure_p'][:] + ds1['pressure_base'][:]
print(pfull.shape)
print(pfull[0,iCell,:])

# Find the top interface level pressure

In [ ]:
from math import log, exp
size = pfull[0,iCell,:].size
logIm1 = ( log(pfull[0,iCell,size-1]) + log(pfull[0,iCell,size-2]) ) /2
logM = log(pfull[0,iCell,size-1])
logI = 2 * logM - logIm1
exp(logI)

## Find and tlot neighborhood cells around a given lat, lon

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from netCDF4 import Dataset
cartopy.config['data_dir'] = f"{pyDAmonitor_ROOT}/data/natural_earth_data"

mylat, mylon0 = 40.14, -105.18   # specify a given lot, lon

lat_incr = 0.01  # degreee, ~1km?
lon_incr = 0.01  # degree, ~1km?

mylon = mylon0 + 360  # convert to 360-based degree
lonCell = np.degrees(ds.variables['lonCell'][:])
latCell = np.degrees(ds.variables['latCell'][:])
lonVertex = np.degrees(ds.variables['lonVertex'][:])
latVertex = np.degrees(ds.variables['latVertex'][:])

# Connectivity: vertices of each cell
verticesOnCell = ds.variables['verticesOnCell'][:]  # shape (nCells, maxEdges)
nEdgesOnCell = ds.variables['nEdgesOnCell'][:]

# find neighborhood cells (>=7)
for iter in range(10):
    latmin = mylat - lat_incr * (iter + 1)
    latmax = mylat + lat_incr * (iter + 1)
    lonmin = mylon - lon_incr * (iter + 1)
    lonmax = mylon + lon_incr * (iter + 1)
    
    maskC = (latCell >= latmin) & (latCell <= latmax) & (lonCell >= lonmin) & (lonCell <= lonmax)
    latC = latCell[maskC]
    lonC = lonCell[maskC]
    vertC = verticesOnCell[maskC]
    nEdgesC = nEdgesOnCell[maskC]
    print(latC.size)
    if latC.size >= 7: 
        break  # exit the loop if we find 7+ cells

# find the cell indices 
iCell = ds0.variables['indexToCellID'][:][maskC]
# ter = ds0.variables["ter"][:][mask]

# --- Plot ---
fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection=ccrs.PlateCarree())

# add costlines, country borders, state/province borders
ax.coastlines(resolution='50m')  # '110m', '50m', or '10m'
ax.add_feature(cfeature.BORDERS, linewidth=0.7)
ax.add_feature(cfeature.STATES, linewidth=0.5, edgecolor='gray')
ax.add_feature(cfeature.LAND, facecolor=cfeature.COLORS['land'])
ax.add_feature(cfeature.OCEAN, facecolor=cfeature.COLORS['water'])

# relax the plotting extent by the "buffer" degree
buffer = 0.02
ax.set_extent([lonmin - buffer, lonmax + buffer, latmin - buffer, latmax + buffer])

# plot ploygons
for i in range(latC.size):
    nEdges = nEdgesC[i]
    verts = verC[i, :nEdges] - 1  # convert 1-based to 0-based index
    poly_lon = lonVertex[verts]
    poly_lat = latVertex[verts]
    ax.fill(poly_lon, poly_lat, edgecolor="black", facecolor="none", linewidth=0.5, transform=ccrs.PlateCarree())
    # add the cell index in the center of the cell
    ax.text(lonC[i], latC[i], f'{iCell[i]}', 
        transform=ccrs.PlateCarree(),
        fontsize=8, color="blue",
        ha="center", va="center")
    
# Plot a star at the user specificed (mylon, mylat)
ax.plot(mylon, mylat, marker="*", color="red", markersize=5,  # o, ^, s
        transform=ccrs.PlateCarree())

plt.show()